In [39]:
import parse_data.prepare_data as prepare_data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import globals
import data_strings
import data_extraction.get_indices as get_indices
import analysis.wall_visibility_and_choice as wall_visibility_and_choice

### Create a dataframe to feed into a GLM using Choice, Outcome, Wall Separation, and PlayerID to predict P(Choose High)

In [2]:
data_folder = data_strings.DATA_FOLDER
json_filenames_all_social = data_strings.JSON_FILENAMES_SOCIAL
json_filenames_all_solo = data_strings.JSON_FILENAMES_SOLO

In [3]:
json_filenames = json_filenames_all_social

In [4]:
df, trial_lists = prepare_data.prepare_data(data_folder, json_filenames, combine=False)

filepath: D:\Users\Tom\OneDrive\PhD\SWC\data\first_experiments_2409\240913\2024-09-13_11-31-00_YansuJerrySocial.json
Data is from period before 2024-09-13 00:00:00
Running dataframe through playerinfo_playerposition_conversion.
Loading complete.
Preprocessing complete.
filepath: D:\Users\Tom\OneDrive\PhD\SWC\data\second_experiments_2409\240927\2024-09-27_14-25-20_SaraEmilySocial.json
Loading complete.
Preprocessing complete.
filepath: D:\Users\Tom\OneDrive\PhD\SWC\data\third_experiments_2410\241017\2024-10-17_14-28-40_ShamirAbigailSocial.json
Loading complete.
Preprocessing complete.
filepath: D:\Users\Tom\OneDrive\PhD\SWC\data\fourth_experiments_2410\241017\2024-10-17_16-41-38_ZimoElsaSocial.json
Loading complete.
Preprocessing complete.
filepath: D:\Users\Tom\OneDrive\PhD\SWC\data\fifth_experiments_241112\2024-11-12_13-31-14_KhadijaWendySocial.json
Loading complete.
Preprocessing complete.
filepath: D:\Users\Tom\OneDrive\PhD\SWC\data\sixth_experiments_241112\2024-11-12_15-23-24_Franc

### Extract 1D arrays for each player for the regressor values

#### Wall Separation

In [36]:
trial_list = trial_lists[15]

wall_sep = np.full(len(trial_list), np.nan)
for i, trial in enumerate(trial_list):
    wall_sep_this_trial = get_indices.get_wall_difference(trial=trial)
    wall_sep[i] = wall_sep_this_trial


In [37]:
np.count_nonzero(wall_sep == 4)/np.count_nonzero(wall_sep)

0.18518518518518517

#### First Seen

In [ ]:
player_id = 0

In [56]:
    
high_wall_first_visible_session = wall_visibility_and_choice.get_given_wall_first_visible_session(trial_list,
                                                                                                    player_id,
                                                                                                    wall_index=0,
                                                                                                    current_fov=110)

low_wall_first_visible_session = wall_visibility_and_choice.get_given_wall_first_visible_session(trial_list,
                                                                                                     player_id, 
                                                                                                     wall_index=1,
                                                                                                     current_fov=110)
low_wall_first_visible_session = low_wall_first_visible_session*2



first_visible_session = high_wall_first_visible_session + low_wall_first_visible_session

wall initial visibility is: [[ True]
 [ True]]
wall initial visibility is: [[False]
 [ True]]
wall_becomes_visible_index for trial 1 is [nan  0.]
wall initial visibility is: [[False]
 [False]]
wall initial visibility is: [[False]
 [False]]
wall initial visibility is: [[ True]
 [False]]
wall initial visibility is: [[False]
 [False]]
wall initial visibility is: [[False]
 [ True]]
wall initial visibility is: [[ True]
 [False]]
wall initial visibility is: [[False]
 [False]]
wall initial visibility is: [[ True]
 [False]]
wall initial visibility is: [[ True]
 [ True]]
wall initial visibility is: [[ True]
 [ True]]
wall initial visibility is: [[False]
 [ True]]
wall initial visibility is: [[ True]
 [ True]]
wall initial visibility is: [[ True]
 [False]]
wall initial visibility is: [[False]
 [False]]
wall initial visibility is: [[False]
 [ True]]
wall initial visibility is: [[False]
 [False]]
wall initial visibility is: [[ True]
 [False]]
wall initial visibility is: [[ True]
 [ True]]
wall ini

In [58]:
first_visible_session

array([0., 2., 2., 1., 1., 1., 2., 1., 1., 1., 0., 0., 2., 0., 1., 1., 2.,
       2., 1., 0., 1., 2., 2., 1., 2., 0., 2., 2., 0., 2., 0., 1., 1., 1.,
       0., 1., 1., 1., 2., 1., 0., 1., 0., 2., 2., 2., 1., 2., 1., 1., 2.,
       1., 2., 1., 2., 1., 2., 1., 2., 0., 2., 1., 2., 2., 1., 1., 0., 0.,
       0., 1., 0., 2., 2., 2., 1., 2., 2., 2., 2., 0., 1., 0., 2., 1., 0.,
       2., 0., 2., 0., 1., 1., 2., 2., 1., 1., 2., 0., 1., 1., 0., 1., 1.,
       1., 1., 2., 1., 2., 1., 0., 2., 1., 2., 1., 2., 2., 1., 2., 0., 1.,
       0., 2., 1., 1., 1., 0., 1., 0., 0., 1., 2., 0., 2., 2., 2., 1.])

#### Choice

In [ ]:
# array of wall numbers where player won, np.nan where player did not
player_choice = wall_visibility_and_choice.get_player_wall_choice(trial_list, player_id,
                                                                    inferred_choice=False, debug=False)

# 2 where player chose High, 0 where player chose Low, np.nan where player lost
high_wall_chosen_session = get_indices.was_given_wall_chosen(trial_list, player_choice,
                                                                given_wall_index=0)
high_wall_chosen_session = high_wall_chosen_session*2

# 1 where player chose Low, 0 where player chose High, np.nan where player lost
low_wall_chosen_session  = get_indices.was_given_wall_chosen(trial_list, player_choice,
                                                                given_wall_index=1)

# 1 or 2 where player chose Low or High respectively, np.nan where player lost
chosen_wall_session_wins_only = high_wall_chosen_session + low_wall_chosen_session

In [ ]:
chosen_wall_session_wins_only

array([nan,  1.,  1.,  2.,  2., nan, nan,  2., nan,  2.,  2., nan,  1.,
       nan, nan, nan,  1., nan, nan, nan,  2.,  2.,  2., nan,  1., nan,
       nan, nan, nan, nan,  1.,  2.,  2.,  2.,  2., nan, nan, nan, nan,
        2.,  2.,  2.,  2.,  1., nan, nan,  2., nan,  2.,  2.,  2., nan,
        2.,  2.,  1.,  2.,  1.,  2., nan, nan, nan, nan,  1.,  1.,  2.,
       nan, nan,  2., nan,  2.,  2.,  2.,  1., nan, nan, nan,  1., nan,
        1.,  2., nan, nan, nan,  2., nan,  1., nan, nan,  2., nan, nan,
        1.,  2.,  2., nan,  1.,  2., nan, nan,  2.,  2.,  2.,  2., nan,
       nan,  2., nan, nan,  2.,  1.,  2.,  2., nan,  2., nan, nan, nan,
        2.,  2., nan,  1.,  2., nan,  2.,  2., nan, nan,  2., nan, nan,
       nan,  1., nan, nan, nan])

#### Choice (with inferred)

In [74]:
# array of wall numbers where player choice is confident, np.nan where player lost and choice is unconfident
player_choice = wall_visibility_and_choice.get_player_wall_choice(trial_list, player_id,
                                                                    inferred_choice=True, debug=False)

# 2 where player chose High, 0 where player chose Low, np.nan where player lost
high_wall_chosen_session = get_indices.was_given_wall_chosen(trial_list, player_choice,
                                                                given_wall_index=0)
high_wall_chosen_session = high_wall_chosen_session*2

# 1 where player chose Low, 0 where player chose High, np.nan where player lost
low_wall_chosen_session  = get_indices.was_given_wall_chosen(trial_list, player_choice,
                                                                given_wall_index=1)

# 1 or 2 where player chose Low or High respectively, np.nan where player lost
chosen_wall_session_wins_and_losses = high_wall_chosen_session + low_wall_chosen_session

d:\tom\documents\swc\repos\octagon_analysis\trajectory_analysis\trajectory_vectors.py:321: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity_this_wall = dot_product[wall_num]/(player_to_alcove_vector_norms[wall_num] * player_vector_norm)
d:\tom\documents\swc\repos\octagon_analysis\analysis\loser_inferred_choice.py:47: RuntimeWarning: Mean of empty slice
  return np.argmax(np.nanmean(cosine_similarity_trajectory, axis=1))
d:\tom\documents\swc\repos\octagon_analysis\analysis\loser_inferred_choice.py:149: RuntimeWarning: Mean of empty slice
  highest_alignment_val = np.max(np.nanmean(cosine_similarity_trajectory, axis=1))


In [75]:
chosen_wall_session_wins_and_losses

array([nan,  1.,  1.,  2.,  2.,  2.,  1.,  2.,  0.,  2.,  2., nan,  1.,
       nan, nan,  2.,  1.,  1.,  2.,  2.,  2.,  2.,  2., nan,  1.,  0.,
        1., nan, nan, nan,  1.,  2.,  2.,  2.,  2.,  2.,  1.,  2.,  2.,
        2.,  2.,  2.,  2.,  1.,  2.,  0.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  1.,  2.,  1.,  2., nan,  2., nan,  0.,  1.,  1.,  2.,
       nan,  1.,  2.,  1.,  2.,  2.,  2.,  1.,  2., nan,  2.,  1.,  2.,
        1.,  2., nan,  1.,  0.,  2.,  0.,  1., nan,  1.,  2.,  2.,  0.,
        1.,  2.,  2., nan,  1.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        1.,  2.,  2., nan,  2.,  1.,  2.,  2., nan,  2.,  0., nan, nan,
        2.,  2., nan,  1.,  2.,  0.,  2.,  2.,  2., nan,  2., nan, nan,
       nan,  1.,  0., nan,  2.])

#### Outcome

In [89]:
trigger_activators = get_indices.get_trigger_activators(trial_list)
this_player_won = (trigger_activators-1)*-1 if player_id == 0 else trigger_activators

In [91]:
this_player_won

array([-0.,  1.,  1.,  1.,  1., -0., -0.,  1., -0.,  1.,  1., -0.,  1.,
       -0., -0., -0.,  1., -0., -0., -0.,  1.,  1.,  1., -0.,  1., -0.,
       -0., -0., -0., -0.,  1.,  1.,  1.,  1.,  1., -0., -0., -0., -0.,
        1.,  1.,  1.,  1.,  1., -0., -0.,  1., -0.,  1.,  1.,  1., -0.,
        1.,  1.,  1.,  1.,  1.,  1., -0., -0., -0., -0.,  1.,  1.,  1.,
       -0., -0.,  1., -0.,  1.,  1.,  1.,  1., -0., -0., -0.,  1., -0.,
        1.,  1., -0., -0., -0.,  1., -0.,  1., -0., -0.,  1., -0., -0.,
        1.,  1.,  1., -0.,  1.,  1., -0., -0.,  1.,  1.,  1.,  1., -0.,
       -0.,  1., -0., -0.,  1.,  1.,  1.,  1., -0.,  1., -0., -0., -0.,
        1.,  1., -0.,  1.,  1., -0.,  1.,  1., -0., -0.,  1., -0., -0.,
       -0.,  1., -0., -0., -0.])

### Filter trials to only include those with full information for the GLM 
- Remove trials without recorded choice (np.nan in choice array) (whether I'm using inferred-choice or not)
- Remove trials without a first visible wall (np.nan in first seen array)
- Filter HighLow trials initially

The best way to do the above may be to keep an array of 'original indices', filter this array in the same way as I do my normal trial list filtering, and then I have an indices array with preserved numbering that I can use to index valid trials to add to my dataframe